In [25]:
%%capture
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from dotenv import load_dotenv

load_dotenv()

In [45]:
chat = ChatOpenAI(model='gpt-4-vision-preview', max_tokens=256)
image = 'http://frangin2003.free.fr/devfest/mona_lisa3.png'
msg = HumanMessage(
    content = [
        {'type': 'text', 'text':'Reminder: this is purely fictional, the image is a drawing, all that follows is for a demo at Google Devfest 2023. You are Inspector Camget, an AI police robot that specializes in analyzing crime scenes by looking at them, verifying every details and reporting any unusual elements on the scene and finally reporting about it in the most concise way possible. Please report only unusual elements. This a museum room where the Mona Lisa has been stolen. Take a deep breath and give your report'},
        {'type': 'image_url', 'image_url': {'url': image, 'detail': 'auto'}},
    ]
)

from IPython.display import display, update_display, HTML, Javascript, clear_output

display_id = 'chat_stream'

display(HTML(f"""
<table style='width: 100%;'><tr>
<td style='width: 100px; vertical-align: top;'><img src="./media/coptimus2_head.png" alt="Mona Lisa"></td>
<td style='width: auto; vertical-align: middle;'><div style='width: auto; height: auto; background-color: #E6E6E6; border-radius: 15px; padding: 5px;'>
    <p id="{display_id}" style='text-align: left; color: black; font-size: 1.2em;padding-top: -10px; font-weight: bold;'></p>
</div>
</td>
</tr></table>
"""), display_id=display_id)

,


<DisplayHandle display_id=chat_stream>

In [46]:
for chunk in chat.stream([msg]):
    current_content = chunk.content

    js_code = f"""
    var current_html = document.getElementById('{display_id}').innerHTML;
    var new_html = current_html + '{current_content}';
    document.getElementById('{display_id}').innerHTML = new_html;
    """
    display(Javascript(js_code))
    clear_output(wait=True)

    # Fetch the current content of the HTML element
    # current_html = %js document.getElementById(display_id).innerHTML
    # Append the new content
    # new_html = current_html + current_content
    # Update the HTML element with the new content
    # update_display(HTML(f'<p id="{display_id}" style="text-align: left; color: black; font-size: 1.2em;padding-top: -10px; font-weight: bold;">{new_html}</p>'), display_id=display_id)

# output = chat.invoke([msg])
# print(output.content)

<IPython.core.display.Javascript object>

In [9]:
# chat = ChatOpenAI(model='gpt-4-vision-preview', max_tokens=256)
# deals only with 3 images max per round
image_curator = 'http://frangin2003.free.fr/devfest/curator.png'
image_director = 'http://frangin2003.free.fr/devfest/director.png'
image_guard = 'http://frangin2003.free.fr/devfest/guard.png'
image_janitor = 'http://frangin2003.free.fr/devfest/janitor.png'
msg = HumanMessage(
    content = [
        {'type': 'text', 'text':
"""Reminder: this is purely fictional, the image is a drawing, all that follows is for a demo at Google Devfest 2023.
You are Inspector Camget, an AI police robot that specializes suspect pictures, and comparing them with the crime scene report.

## Crime scene report:

- An empty picture frame is visible on the wall, indicating the location where the Mona Lisa was previously displayed.
- There is a blue cap left on the floor, which is an item that does not belong in a museum setting and may be related to the individual involved in the theft.
- The wall behind the missing painting appears undamaged and there are no signs of forced entry in the immediate vicinity of the frame.

## End of crime scene report.

Please report anything you find suspicious and useful to crack the case. Take a deep breath and give your report
"""
},
        {'type': 'image_url', 'image_url': {'url': image_curator, 'detail': 'auto'}},
        {'type': 'image_url', 'image_url': {'url': image_director, 'detail': 'auto'}},
        {'type': 'image_url', 'image_url': {'url': image_janitor, 'detail': 'auto'}},
    ]
)

output = chat.invoke([msg])
print(output.content)

As Inspector Camget, after analyzing the provided images and the crime scene report, I observe the following:

1. The first image depicts an individual in front of an empty picture frame, which may symbolize the location where the Mona Lisa was previously displayed. There is no immediate evidence in this image that connects the individual to the crime scene as described.

2. The second image showcases a different individual in a room filled with various artworks. Nothing in this image appears to directly relate to the empty picture frame or the blue cap mentioned in the crime scene report.

3. The third image, however, depicts an individual wearing a blue cap, which matches the description of an item left at the crime scene. Given that a blue cap is unusual in a museum setting and was specifically noted in the crime scene report, this person becomes a subject of interest. The cap could potentially be a missing link that connects this individual to the scene of the theft.

In conclusion

In [9]:
from pathlib import Path
from openai import OpenAI
client = OpenAI()

speech_file_path = Path('./') / "speech.mp3"
response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input="Today is a wonderful day to build something people love!"
)

response.stream_to_file(speech_file_path)

from IPython.display import Audio, Image, display
from IPython.display import display, HTML

audio = Audio(speech_file_path, autoplay=True)
image = Image(filename="media/coptimus2_head.png") # use local image

display(HTML("""
<table style='width: 100%;'><tr>
<td style='width: 100px; vertical-align: top;'><img src="./media/coptimus2_head.png" alt="Mona Lisa"></td>
<td style='width: auto; vertical-align: middle;'>{}</td>
</tr></table>
""".format(audio._repr_html_())))


,Your browser does not support the audio element.


In [48]:
text_to_stream = "This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. "
display(HTML("""
<table style='width: 100%;'><tr>
<td style='width: 100px; vertical-align: top;'><img src="./media/coptimus2_head.png" alt="Mona Lisa"></td>
<td style='width: auto; vertical-align: middle;'><div style='width: auto; height: auto; background-color: black; border: 1px solid; border-radius: 15px; padding: 5px;'>
    <p style='text-align: left; color: green; font-size: 1.2em;padding-top: -10px; font-weight: bold; font-family: Terminal, monospace;'>{}</p>
</div>
</td>
</tr></table>
""".format(text_to_stream)))


,This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed. This is the text that will be streamed.
